# Trainingspipeline 4

- ohne Bigramme
- vector_size: 300
- window: 15
- min_count: 2
- seed: 1

In [1]:
import codecs
import nltk
import numpy as np
import os
import pandas as pd
import re
import scipy
import spacy

from gensim.models import Word2Vec
from joblib import Parallel, delayed  
from nltk.corpus import stopwords

In [2]:
stopwords = stopwords.words('italian')
tokenizer = nltk.data.load('tokenizers/punkt/italian.pickle')

In [3]:
df = pd.read_csv('../../Korpus/Korpus/corpus_lemmatized.csv', sep=',', encoding='utf-8')

In [5]:
df.shape

(304129, 10)

In [6]:
df.text = df.text.fillna('')
df.lemmatized_text = df.lemmatized_text.fillna('')

In [7]:
# Einzeldataframes für die Zeiträume

df_periods = dict(tuple(df.groupby(by='period')))

In [8]:
df1 = df_periods['1700-1750']
df2 = df_periods['1751-1800']
df3 = df_periods['1801-1825']
df4 = df_periods['1826-1850']
df5 = df_periods['1851-1875']
df6 = df_periods['1876-1900']
df7 = df_periods['1901-1925']
df8 = df_periods['1926-1950']
df9 = df_periods['1951-1985']
df10 = df_periods['1986-2000']
df11 = df_periods['2001-2021']

## Training von Word2Vec

In [9]:
# Hilfsfunktionen zur Vorbereitung auf das Training
# Bereinigung und Tokenisierung

def sentence_to_wordlist(raw:str):
    """
    cleans and tokenizes the sentences
    """
    text = re.sub('[^A-Za-z_àÀèÈìÌòÒùÙáÁéÉíÍóÓúÚ]',' ', raw).split()        # Diakritika ans Italienische anpassen                    
    filtered_text = [word for word in text if word not in stopwords]        # Stopwörter löschen
    return filtered_text


def tokenize_text(raw_text):
    """
    returns a list of lowercase tokenized sentences 
    """
    raw_sentences = tokenizer.tokenize(str(raw_text).lower())    
    tokenized_sentences = Parallel(n_jobs=-1)(delayed(sentence_to_wordlist)(raw_sentence) for raw_sentence in raw_sentences)
    sentences = tokenized_sentences
    return sentences

In [10]:
# Trainingsparamter setzen

vector_size = 300                  # Dimensionality of the word vectors
window = 15                        # The maximum distance between the current and predicted word within a sentence
min_count = 2                      # (int, optional) – The model ignores all words with total frequency lower than this
workers = 1                        # Use these many worker threads to train the model (faster training with multicore machines)
min_alpha = 0.0001                 # Learning rate will linearly drop to min_alpha as training progresses
sg = 1                             # Training algorithm: skip-gram if sg=1, otherwise CBOW            
seed = 1                           # Reproductivity --> only if workers = 1

### Zeitraum 1: 1700-1750

In [11]:
# lemmatisierte Texte zu einem String verbinden

text1 = ''

for i in df1.lemmatized_text:
    text1 += i

In [12]:
%%time
sentences1 = tokenize_text(text1)         # Bereinigen, Tokenisieren und in Form bringen (Ziel: Liste von tokenisierten Sätzen)

Wall time: 18.6 s


In [13]:
print(sentences1[3])

['lungo', 'inospita', 'campagna', 'aggiungere', 'invidia', 'lato', 'manco', 'dire', 'anch', 'essere', 'teco', 'labbro', 'bianco', 'veggo', 'veneno', 'cor', 'stagno']


In [14]:
len(sentences1)

113328

In [15]:
%%time

# Training   

w2v1 = Word2Vec(sentences=sentences1,                      
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 3min 50s


In [16]:
w2v1.wv.most_similar(positive=['terrore'], topn=10)

[('riempiè', 0.6970417499542236),
 ('trucidare', 0.693036675453186),
 ('ribellate', 0.667370080947876),
 ('vitellio', 0.6592564582824707),
 ('macello', 0.6589673757553101),
 ('smantellare', 0.6562298536300659),
 ('alamanni', 0.6539049744606018),
 ('sconfiggere', 0.652942955493927),
 ('spavento', 0.6523786187171936),
 ('impadronitosi', 0.6523432731628418)]

In [17]:
# trainiertes Modell speichern

w2v1.save(os.path.join('trained_models/Word2Vec4', '4w2v1.model'))

### Zeitraum 2: 1751-1800

In [18]:
text2 = ''

for i in df2.lemmatized_text:
    text2 += i

In [19]:
%%time
sentences2 = tokenize_text(text2)

Wall time: 19.9 s


In [20]:
%%time
w2v2 = Word2Vec(sentences=sentences2,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 3min 22s


In [21]:
w2v2.wv.most_similar(positive=['terrore'], topn=10)

[('atterrire', 0.8028432726860046),
 ('fantasma', 0.7545130252838135),
 ('sacerdoti', 0.7539050579071045),
 ('codardia', 0.7518543004989624),
 ('scompiglio', 0.7487813830375671),
 ('respingere', 0.747591495513916),
 ('attentato', 0.7449971437454224),
 ('sfrenato', 0.7423592805862427),
 ('usurpatore', 0.7402671575546265),
 ('assalto', 0.7393408417701721)]

In [22]:
w2v2.save(os.path.join('trained_models/Word2Vec4', '4w2v2.model'))

### Zeitraum 3: 1801-1825

In [23]:
text3 = ''

for i in df3.lemmatized_text:
    text3 += i

In [24]:
%%time
sentences3 = tokenize_text(text3)

Wall time: 15.6 s


In [25]:
%%time
w2v3 = Word2Vec(sentences=sentences3,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 3min 34s


In [26]:
w2v3.wv.most_similar(positive=['terrore'], topn=10)

[('violentare', 0.7314805388450623),
 ('simulazione', 0.7302935719490051),
 ('abbattimento', 0.7235227823257446),
 ('agguerrire', 0.7232373356819153),
 ('agitarla', 0.7218489646911621),
 ('reprimere', 0.7165923118591309),
 ('avvilire', 0.7163895964622498),
 ('prorompere', 0.708271861076355),
 ('fraudi', 0.7058936357498169),
 ('comprometterti', 0.7045263051986694)]

In [27]:
w2v3.save(os.path.join('trained_models/Word2Vec4', '4w2v3.model'))

### Zeitraum 4: 1826-1850

In [28]:
text4 = ''

for i in df4.lemmatized_text:
    text4 += i

In [29]:
%%time
sentences4 = tokenize_text(text4)

Wall time: 15.7 s


In [30]:
%%time
w2v4 = Word2Vec(sentences=sentences4,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 3min 30s


In [31]:
w2v4.wv.most_similar(positive=['terrore'], topn=10)

[('opprimere', 0.7404669523239136),
 ('spavento', 0.7001821994781494),
 ('malcontenti', 0.6927286386489868),
 ('indegnazione', 0.6796774864196777),
 ('atterrire', 0.6794615387916565),
 ('scompiglio', 0.675117015838623),
 ('incitare', 0.6564921140670776),
 ('spaventare', 0.6553658246994019),
 ('furore', 0.6543145179748535),
 ('impetuoso', 0.6504160761833191)]

In [32]:
w2v4.save(os.path.join('trained_models/Word2Vec4', '4w2v4.model'))

### Zeitraum 5: 1851-1875

In [33]:
text5 = ''

for i in df5.lemmatized_text:
    text5 += i

In [34]:
%%time
sentences5 = tokenize_text(text5)

Wall time: 16.7 s


In [35]:
%%time
w2v5 = Word2Vec(sentences=sentences5,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 3min 12s


In [36]:
w2v5.wv.most_similar(positive=['terrore'], topn=10)

[('terribil', 0.750889241695404),
 ('spavento', 0.7356125712394714),
 ('straziante', 0.7285497188568115),
 ('opprimere', 0.7274367809295654),
 ('sopraffare', 0.7271804213523865),
 ('agonia', 0.7171481847763062),
 ('saettò', 0.7158385515213013),
 ('fremito', 0.7133640050888062),
 ('brivido', 0.7116336226463318),
 ('svenire', 0.7102494239807129)]

In [37]:
w2v5.save(os.path.join('trained_models/Word2Vec4', '4w2v5.model'))

### Zeitraum 6: 1876-1900

In [38]:
text6 = ''

for i in df6.lemmatized_text:
    text6 += i

In [39]:
%%time
sentences6 = tokenize_text(text6)

Wall time: 18.3 s


In [40]:
%%time
w2v6 = Word2Vec(sentences=sentences6,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 57s


In [41]:
w2v6.wv.most_similar(positive=['terrore'], topn=10)

[('sgomento', 0.7740588188171387),
 ('incutere', 0.715033769607544),
 ('angoscioso', 0.708915114402771),
 ('invasare', 0.7055569887161255),
 ('parossismo', 0.6964709162712097),
 ('spaventoso', 0.6906256675720215),
 ('spavento', 0.684069812297821),
 ('angoscia', 0.6782659888267517),
 ('orrore', 0.6779136061668396),
 ('crollo', 0.677290141582489)]

In [42]:
w2v6.save(os.path.join('trained_models/Word2Vec4', '4w2v6.model'))

### Zeitraum 7: 1901-1925

In [43]:
text7 = ''

for i in df7.lemmatized_text:
    text7 += i

In [44]:
%%time
sentences7 = tokenize_text(text7)

Wall time: 17.7 s


In [45]:
%%time
w2v7 = Word2Vec(sentences=sentences7,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 3min 1s


In [46]:
w2v7.wv.most_similar(positive=['terrore'], topn=10)

[('subitaneo', 0.722050666809082),
 ('orrore', 0.6921170353889465),
 ('sdegno', 0.6732694506645203),
 ('indicibile', 0.670876145362854),
 ('bastiglia', 0.6707304120063782),
 ('scolorare', 0.6649217009544373),
 ('smarrimento', 0.6634646654129028),
 ('indignazione', 0.6599095463752747),
 ('repugnanza', 0.6590630412101746),
 ('invasare', 0.6565138697624207)]

In [47]:
w2v7.save(os.path.join('trained_models/Word2Vec4', '4w2v7.model'))

### Zeitraum 8: 1926-1950

In [48]:
text8 = ''

for i in df8.lemmatized_text:
    text8 += i

In [49]:
%%time
sentences8 = tokenize_text(text8)

Wall time: 17 s


In [50]:
%%time
w2v8 = Word2Vec(sentences=sentences8,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 3min 14s


In [51]:
w2v8.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.7136622667312622),
 ('contratte', 0.6966002583503723),
 ('rabbico', 0.6928951740264893),
 ('virus', 0.692168653011322),
 ('subitaneo', 0.692112386226654),
 ('orrendo', 0.6918310523033142),
 ('ribrezzo', 0.690100371837616),
 ('raccapriccio', 0.6848952770233154),
 ('tremito', 0.6837735772132874),
 ('massimiliana', 0.6774007678031921)]

In [52]:
w2v8.save(os.path.join('trained_models/Word2Vec4', '4w2v8.model'))

### Zeitraum 9: 1951-1985

In [53]:
text9 = ''

for i in df9.lemmatized_text:
    text9 += i

In [54]:
%%time
sentences9 = tokenize_text(text9)

Wall time: 17.7 s


In [55]:
%%time
w2v9 = Word2Vec(sentences=sentences9,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 3min 18s


In [56]:
w2v9.wv.most_similar(positive=['terrore'], topn=10)

[('annientare', 0.6422019600868225),
 ('scampo', 0.617358386516571),
 ('sconvolgere', 0.604812741279602),
 ('valanga', 0.6043187379837036),
 ('sanguinoso', 0.599715530872345),
 ('impossessare', 0.5987902879714966),
 ('spirale', 0.5984514355659485),
 ('liberatorio', 0.5965858697891235),
 ('involare', 0.5959235429763794),
 ('latinoamericano', 0.5933594107627869)]

In [57]:
w2v9.save(os.path.join('trained_models/Word2Vec4', '4w2v9.model'))

### Zeitraum 10: 1986-2000

In [58]:
text10 = ''

for i in df10.lemmatized_text:
    text10+= i

In [59]:
%%time
sentences10 = tokenize_text(text10)

Wall time: 39.3 s


In [60]:
%%time
w2v10 = Word2Vec(sentences=sentences10,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 3min 17s


In [61]:
w2v10.wv.most_similar(positive=['terrore'], topn=10)

[('insanguinare', 0.7149465680122375),
 ('sopravvissuto', 0.7143348455429077),
 ('hitleriano', 0.7027631998062134),
 ('seminare', 0.6948736906051636),
 ('brutalità', 0.6921743750572205),
 ('lager', 0.6884458065032959),
 ('incutere', 0.6879408359527588),
 ('inneggiare', 0.683692455291748),
 ('affogare', 0.6807557344436646),
 ('macchiare', 0.6804212331771851)]

In [62]:
w2v10.save(os.path.join('trained_models/Word2Vec4', '4w2v10.model'))

### Zeitraum 11: 2001-2021

In [63]:
text11 = ''

for i in df11.lemmatized_text:
    text11+= i

In [64]:
%%time
sentences11 = tokenize_text(text11)

Wall time: 12.4 s


In [65]:
%%time
w2v11 = Word2Vec(sentences=sentences11,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 4min 39s


In [66]:
w2v11.wv.most_similar(positive=['terrore'], topn=10)

[('seminare', 0.8013615012168884),
 ('orrore', 0.7996792197227478),
 ('distruzione', 0.7798026204109192),
 ('crudele', 0.7782591581344604),
 ('atrocità', 0.7755215167999268),
 ('empio', 0.7727988362312317),
 ('terrorizzare', 0.7656313180923462),
 ('allah', 0.7572340369224548),
 ('devastante', 0.7566092014312744),
 ('atrocemente', 0.755970299243927)]

In [67]:
w2v11.save(os.path.join('trained_models/Word2Vec4', '4w2v11.model'))